In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
tf.keras.backend.clear_session()

In [5]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [6]:
subject = 'Roraima - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [80]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_RR.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data = data.drop('Roraima - Produção de Cimento (t)', axis=1)
data

,Ano,Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - IDH,Roraima - IDH Longevidade,Roraima - IDH Educacao,Roraima - IDH Renda,Contratação Comercial (Bi R$),...,Taxa Selic (%),Roraima - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Roraima - Consumo de Cimento (t)
0,2008,6.555833e+06,342849.112404,12.235418,5.474033e+06,0.736454,0.752834,0.733615,0.722576,5.483335e+08,...,11.760983,8.015090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.009000
1,2009,6.611657e+06,345335.485288,12.228462,5.482306e+06,0.736503,0.754681,0.734851,0.720114,5.302764e+08,...,9.472585,8.046321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.064000
2,2010,6.787712e+06,348962.483930,12.215749,5.534487e+06,0.736471,0.756148,0.735694,0.718135,5.046425e+08,...,9.332485,8.009229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.408000
3,2011,6.947966e+06,347630.972013,12.206783,5.574553e+06,0.736360,0.757242,0.736139,0.716628,4.854622e+08,...,10.985689,7.996819,22.217634,1001.541199,1506.456126,27.425647,21.330889,71.595088,1.830447,103.009000
4,2012,7.096137e+06,347411.232273,12.186539,5.609510e+06,0.736089,0.757859,0.736249,0.715498,4.867661e+08,...,8.159014,8.651837,21.157249,1001.690342,1515.527356,27.608612,21.117524,70.061650,1.898255,100.066000
5,2013,7.239848e+06,349584.100654,12.182243,5.648192e+06,0.735929,0.758347,0.735436,0.715372,5.225848e+08,...,7.913561,9.551098,18.013769,1001.817868,1528.998642,27.762053,20.876766,68.552790,1.943125,121.914000
6,2014,7.349320e+06,344289.736627,12.154911,5.667961e+06,0.735757,0.758460,0.735472,0.714737,5.676413e+08,...,10.350442,10.328300,14.644773,1001.940536,1542.131372,27.986329,20.647922,66.896135,1.956684,120.417571
7,2015,7.439258e+06,340498.356490,12.130631,5.685185e+06,0.735070,0.758081,0.733678,0.714965,5.179392e+08,...,12.454398,10.723767,19.007683,1001.988306,1541.272845,28.048825,20.665808,66.738045,1.910770,113.282057
8,2016,7.513614e+06,340512.038701,12.120014,5.705074e+06,0.735168,0.766432,0.738349,0.702779,4.524144e+08,...,13.111908,10.764101,24.437747,1002.055065,1514.905960,27.959130,20.665948,67.018448,1.902067,105.481371
9,2017,7.560265e+06,340005.357917,12.115483,5.723617e+06,0.733637,0.765744,0.735380,0.701759,4.213689e+08,...,9.479812,10.533542,26.318142,1002.155784,1495.611624,27.867978,20.547545,66.903661,1.943183,99.723000


In [81]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - IDH,Roraima - IDH Longevidade,Roraima - IDH Educacao,Roraima - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Roraima - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,6.555833e+06,342849.112404,12.235418,5.474033e+06,0.736454,0.752834,0.733615,0.722576,5.483335e+08,2.546644e+09,...,2.868108,11.760983,8.015090,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6.611657e+06,345335.485288,12.228462,5.482306e+06,0.736503,0.754681,0.734851,0.720114,5.302764e+08,2.679945e+09,...,2.305885,9.472585,8.046321,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6.787712e+06,348962.483930,12.215749,5.534487e+06,0.736471,0.756148,0.735694,0.718135,5.046425e+08,2.891930e+09,...,1.958246,9.332485,8.009229,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6.947966e+06,347630.972013,12.206783,5.574553e+06,0.736360,0.757242,0.736139,0.716628,4.854622e+08,3.091988e+09,...,2.079088,10.985689,7.996819,22.217634,1001.541199,1506.456126,27.425647,21.330889,71.595088,1.830447
4,7.096137e+06,347411.232273,12.186539,5.609510e+06,0.736089,0.757859,0.736249,0.715498,4.867661e+08,3.189506e+09,...,2.183282,8.159014,8.651837,21.157249,1001.690342,1515.527356,27.608612,21.117524,70.061650,1.898255
5,7.239848e+06,349584.100654,12.182243,5.648192e+06,0.735929,0.758347,0.735436,0.715372,5.225848e+08,3.093872e+09,...,2.403445,7.913561,9.551098,18.013769,1001.817868,1528.998642,27.762053,20.876766,68.552790,1.943125
6,7.349320e+06,344289.736627,12.154911,5.667961e+06,0.735757,0.758460,0.735472,0.714737,5.676413e+08,2.779340e+09,...,2.748397,10.350442,10.328300,14.644773,1001.940536,1542.131372,27.986329,20.647922,66.896135,1.956684
7,7.439258e+06,340498.356490,12.130631,5.685185e+06,0.735070,0.758081,0.733678,0.714965,5.179392e+08,2.561731e+09,...,2.501529,12.454398,10.723767,19.007683,1001.988306,1541.272845,28.048825,20.665808,66.738045,1.910770
8,7.513614e+06,340512.038701,12.120014,5.705074e+06,0.735168,0.766432,0.738349,0.702779,4.524144e+08,2.516859e+09,...,2.073464,13.111908,10.764101,24.437747,1002.055065,1514.905960,27.959130,20.665948,67.018448,1.902067
9,7.560265e+06,340005.357917,12.115483,5.723617e+06,0.733637,0.765744,0.735380,0.701759,4.213689e+08,2.497710e+09,...,2.177386,9.479812,10.533542,26.318142,1002.155784,1495.611624,27.867978,20.547545,66.903661,1.943183


In [82]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      63.064000
1      86.408000
2     103.009000
3     100.066000
4     121.914000
5     120.417571
6     113.282057
7     105.481371
8      99.723000
9      83.012000
10    121.120000
11    151.571000
12    144.989000
13    132.217000
14           NaN
Name: Roraima - Consumo de Cimento (t), dtype: float64

In [83]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input = train_input.fillna(0.0)
train_input

,Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - IDH,Roraima - IDH Longevidade,Roraima - IDH Educacao,Roraima - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Roraima - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.744850,-0.315007,1.401150,-1.746144,0.761548,-1.560951,-0.502066,1.373638,1.204752,-0.990234,...,2.026352,0.984445,-1.243205,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-1.589344,0.383838,1.268386,-1.652750,0.786997,-1.123635,0.058230,1.051274,0.820657,-0.546688,...,0.085558,-0.051710,-1.216598,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,-1.098918,1.403279,1.025732,-1.063622,0.770203,-0.776474,0.440255,0.792039,0.275392,0.158672,...,-1.114490,-0.115145,-1.248198,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,-0.652507,1.029031,0.854618,-0.611278,0.712391,-0.517408,0.641649,0.594618,-0.132596,0.824346,...,-0.697344,0.633403,-1.258770,0.026787,-1.750903,0.264474,-1.694225,2.033190,1.942658,-2.299835
4,-0.239756,0.967268,0.468245,-0.216621,0.570378,-0.371368,0.691446,0.446582,-0.104860,1.148829,...,-0.337667,-0.646476,-0.700745,-0.229627,-1.027431,0.358296,-1.009920,1.276809,1.179290,-0.506518
5,0.160572,1.577997,0.386241,0.220096,0.487127,-0.255786,0.323198,0.430184,0.657046,0.830615,...,0.422337,-0.757614,0.065357,-0.989758,-0.408818,0.497626,-0.436039,0.423317,0.428157,0.680157
6,0.465520,0.089910,-0.135410,0.443290,0.396938,-0.229205,0.339660,0.346932,1.615453,-0.215960,...,1.613108,0.345772,0.727473,-1.804421,0.186229,0.633454,0.402771,-0.387939,-0.396551,1.038753
7,0.716056,-0.975734,-0.598807,0.637745,0.037917,-0.318769,-0.473299,0.376803,0.558230,-0.940033,...,0.760923,1.298413,1.064381,-0.749418,0.417955,0.624574,0.636511,-0.324530,-0.475251,-0.175540
8,0.923184,-0.971888,-0.801440,0.862290,0.089103,1.657894,1.643124,-1.219209,-0.835563,-1.089341,...,-0.716758,1.596124,1.098742,0.563636,0.741797,0.351869,0.301046,-0.324036,-0.335662,-0.405692
9,1.053138,-1.114301,-0.887917,1.071645,-0.710639,1.495023,0.298008,-1.352812,-1.495939,-1.153056,...,-0.358019,-0.048438,0.902324,1.018338,1.230371,0.152313,-0.039871,-0.743775,-0.392804,0.681701


In [84]:
reshaped_train_input = dfToInputRNN(train_input)

In [85]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 0.90685251, -1.00761554, -1.41116178,  0.9194875 ,
         -1.74185276,  0.80148276, -1.54447744, -1.37431419,
         -1.0453709 ,  1.21859064, -0.99976018,  0.79428371,
          1.15575749, -0.52559858,  1.30260559, -0.55253046,
         -0.25130359,  1.38787176,  1.47154451, -1.80649616,
          1.09296291,  0.94281138,  0.52836084,  0.61258103,
          0.87656257,  1.47801536,  1.71982086,  1.28221181,
          1.39086472,  1.58209844,  1.08872091,  0.55212962,
         -1.81398118,  1.44113674,  1.11191094,  1.30470835,
          0.86390516, -1.67365968, -1.70738939, -1.55813196,
         -0.48498301, -0.75714639,  0.40770893, -0.27394734,
          2.02323219,  1.2908951 ,  1.70465513,  1.81148084,
          1.39968653, -1.36191442,  1.28024593,  1.11054795,
          0.12662799,  1.40414112,  1.40468999,  1.45643959,
          1.58509336,  1.32283157,  1.30222047,  1.55138602,
          1.91801602, -0.77489784,  2.32098884, -1.1942541 ,
         -1.17081042,  0

In [86]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0      63.064000
1      86.408000
2     103.009000
3     100.066000
4     121.914000
5     120.417571
6     113.282057
7     105.481371
8      99.723000
9      83.012000
10    121.120000
11    151.571000
Name: Roraima - Consumo de Cimento (t), dtype: float64

In [87]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    151.571
Name: Roraima - Consumo de Cimento (t), dtype: float64

In [88]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - IDH,Roraima - IDH Longevidade,Roraima - IDH Educacao,Roraima - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Roraima - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.646085,-0.836784,-1.404774,0.642293,-2.414957,0.237678,-2.61321,-1.325075,1.108405,2.635823,...,-1.482779,-2.260447,0.345514,-1.395304,-1.927921,-2.672061,2.471949,-1.925114,-2.328206,2.648601


In [89]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 0.64608546, -0.83678428, -1.40477375,  0.64229273,
         -2.41495677,  0.23767786, -2.61321023, -1.32507497,
          1.10840491,  2.63582309, -0.18995911,  0.72640489,
          2.66833308, -0.032632  ,  0.98802435,  0.24980494,
         -0.08955808,  1.85434394,  2.02706568, -2.21150703,
          1.42938591,  0.85132668,  0.14092386,  0.30902324,
          1.00267904,  2.27970137,  1.11272593,  0.61007902,
          0.89064538,  1.42231438,  1.02650562,  1.47127593,
         -1.53201473,  1.73503858,  1.66610613,  1.0456894 ,
          0.55289259, -1.81216593, -2.10342296, -2.55377478,
          0.41940628, -1.75756445,  0.241023  , -0.98659345,
          1.52184916,  1.01398607,  1.49290967,  1.40764092,
          1.87930411, -2.51892072,  2.83356214,  1.58620924,
         -0.02524128,  1.54231542,  1.5176119 ,  1.7355269 ,
          1.87941982,  1.54198129,  1.51276712,  1.83795594,
          2.62210693,  1.04993174,  0.96476493, -1.48277861,
         -2.26044738,  0

In [90]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    144.989
Name: Roraima - Consumo de Cimento (t), dtype: float64

In [91]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=1000, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True)
    
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh', return_sequences=True)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [92]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(5)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [93]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2750205401, 101428595, 2278151065, 3140603278, 3774595860]


Step: 0 ___________________________________________
loss: 2.14557147026062
winner_seed: 2750205401


Step: 1 ___________________________________________
loss: 2.209552526473999


Step: 2 ___________________________________________
loss: 1.994192123413086
winner_seed: 2278151065


Step: 3 ___________________________________________
loss: 1.7249336242675781
winner_seed: 3140603278


Step: 4 ___________________________________________
loss: 3.052579879760742


final_seed: 3140603278


In [94]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 3s 3s/step - loss: 11656.3906
Epoch 2/10000
1/1 [==============================] - 0s 8ms/step - loss: 11396.0078
Epoch 3/10000
1/1 [==============================] - 0s 7ms/step - loss: 11133.7178
Epoch 4/10000
1/1 [==============================] - 0s 7ms/step - loss: 10847.5488
Epoch 5/10000
1/1 [==============================] - 0s 7ms/step - loss: 10488.8203
Epoch 6/10000
1/1 [==============================] - 0s 7ms/step - loss: 10046.5586
Epoch 7/10000
1/1 [==============================] - 0s 8ms/step - loss: 9293.2773
Epoch 8/10000
1/1 [==============================] - 0s 7ms/step - loss: 7973.8140
Epoch 9/10000
1/1 [==============================] - 0s 7ms/step - loss: 5640.0015
Epoch 10/10000
1/1 [==============================] - 0s 7ms/step - loss: 2642.9570
Epoch 11/10000
1/1 [==============================] - 0s 7ms/step - loss: 648.8621
Epoch 12/10000
1/1 [==============================] - 0s 7ms/step - loss: 427.494

1/1 [==============================] - 0s 4ms/step - loss: 21.6289
Epoch 101/10000
1/1 [==============================] - 0s 4ms/step - loss: 51.6041
Epoch 102/10000
1/1 [==============================] - 0s 4ms/step - loss: 74.2956
Epoch 103/10000
1/1 [==============================] - 0s 4ms/step - loss: 31.8872
Epoch 104/10000
1/1 [==============================] - 0s 4ms/step - loss: 53.0360
Epoch 105/10000
1/1 [==============================] - 0s 4ms/step - loss: 19.4710
Epoch 106/10000
1/1 [==============================] - 0s 4ms/step - loss: 37.9138
Epoch 107/10000
1/1 [==============================] - 0s 5ms/step - loss: 83.3887
Epoch 108/10000
1/1 [==============================] - 0s 4ms/step - loss: 30.5195
Epoch 109/10000
1/1 [==============================] - 0s 4ms/step - loss: 43.1390
Epoch 110/10000
1/1 [==============================] - 0s 5ms/step - loss: 36.5524
Epoch 111/10000
1/1 [==============================] - 0s 4ms/step - loss: 32.2447
Epoch 112/10000
1/1 

1/1 [==============================] - 0s 4ms/step - loss: 21.7829
Epoch 199/10000
1/1 [==============================] - 0s 4ms/step - loss: 23.8693
Epoch 200/10000
1/1 [==============================] - 0s 4ms/step - loss: 53.1397
Epoch 201/10000
1/1 [==============================] - 0s 4ms/step - loss: 28.5880
Epoch 202/10000
1/1 [==============================] - 0s 4ms/step - loss: 39.5000
Epoch 203/10000
1/1 [==============================] - 0s 4ms/step - loss: 35.1390
Epoch 204/10000
1/1 [==============================] - 0s 4ms/step - loss: 15.0595
Epoch 205/10000
1/1 [==============================] - 0s 4ms/step - loss: 34.4168
Epoch 206/10000
1/1 [==============================] - 0s 4ms/step - loss: 18.7993
Epoch 207/10000
1/1 [==============================] - 0s 4ms/step - loss: 28.6641
Epoch 208/10000
1/1 [==============================] - 0s 4ms/step - loss: 60.8622
Epoch 209/10000
1/1 [==============================] - 0s 4ms/step - loss: 56.5826
Epoch 210/10000
1/1 

1/1 [==============================] - 0s 4ms/step - loss: 32.9518
Epoch 297/10000
1/1 [==============================] - 0s 4ms/step - loss: 25.7518
Epoch 298/10000
1/1 [==============================] - 0s 4ms/step - loss: 21.3963
Epoch 299/10000
1/1 [==============================] - 0s 5ms/step - loss: 24.7287
Epoch 300/10000
1/1 [==============================] - 0s 4ms/step - loss: 62.7454
Epoch 301/10000
1/1 [==============================] - 0s 4ms/step - loss: 19.8761
Epoch 302/10000
1/1 [==============================] - 0s 4ms/step - loss: 14.5288
Epoch 303/10000
1/1 [==============================] - 0s 4ms/step - loss: 15.4100
Epoch 304/10000
1/1 [==============================] - 0s 8ms/step - loss: 9.5914
Epoch 305/10000
1/1 [==============================] - 0s 4ms/step - loss: 17.4717
Epoch 306/10000
1/1 [==============================] - 0s 4ms/step - loss: 34.8309
Epoch 307/10000
1/1 [==============================] - 0s 4ms/step - loss: 21.7193
Epoch 308/10000
1/1 [

Epoch 395/10000
1/1 [==============================] - 0s 4ms/step - loss: 7.9655
Epoch 396/10000
1/1 [==============================] - 0s 5ms/step - loss: 13.3704
Epoch 397/10000
1/1 [==============================] - 0s 4ms/step - loss: 29.3314
Epoch 398/10000
1/1 [==============================] - 0s 4ms/step - loss: 20.7745
Epoch 399/10000
1/1 [==============================] - 0s 5ms/step - loss: 16.8842
Epoch 400/10000
1/1 [==============================] - 0s 5ms/step - loss: 28.8829
Epoch 401/10000
1/1 [==============================] - 0s 4ms/step - loss: 20.5022
Epoch 402/10000
1/1 [==============================] - 0s 5ms/step - loss: 31.3651
Epoch 403/10000
1/1 [==============================] - 0s 4ms/step - loss: 36.9017
Epoch 404/10000
1/1 [==============================] - 0s 4ms/step - loss: 12.3945
Epoch 405/10000
1/1 [==============================] - 0s 4ms/step - loss: 29.2012
Epoch 406/10000
1/1 [==============================] - 0s 4ms/step - loss: 15.5938
Epoch

...

1/1 [==============================] - 0s 4ms/step - loss: 8.6531
Epoch 5184/10000
1/1 [==============================] - 0s 5ms/step - loss: 11.4785
Epoch 5185/10000
1/1 [==============================] - 0s 4ms/step - loss: 10.1370
Epoch 5186/10000
1/1 [==============================] - 0s 4ms/step - loss: 7.3443
Epoch 5187/10000
1/1 [==============================] - 0s 4ms/step - loss: 24.3452
Epoch 5188/10000
1/1 [==============================] - 0s 4ms/step - loss: 12.9209
Epoch 5189/10000
1/1 [==============================] - 0s 4ms/step - loss: 12.4983
Epoch 5190/10000
1/1 [==============================] - 0s 4ms/step - loss: 10.3705
Epoch 5191/10000
1/1 [==============================] - 0s 5ms/step - loss: 11.3591
Epoch 5192/10000
1/1 [==============================] - 0s 4ms/step - loss: 7.9832
Epoch 5193/10000
1/1 [==============================] - 0s 4ms/step - loss: 8.1090
Epoch 5194/10000
1/1 [==============================] - 0s 4ms/step - loss: 12.2466
Epoch 5195/10

1/1 [==============================] - 0s 4ms/step - loss: 17.0240
Epoch 5282/10000
1/1 [==============================] - 0s 4ms/step - loss: 11.6060
Epoch 5283/10000
1/1 [==============================] - 0s 4ms/step - loss: 9.2631
Epoch 5284/10000
1/1 [==============================] - 0s 5ms/step - loss: 13.1516
Epoch 5285/10000
1/1 [==============================] - 0s 4ms/step - loss: 11.4964
Epoch 5286/10000
1/1 [==============================] - 0s 4ms/step - loss: 21.4999
Epoch 5287/10000
1/1 [==============================] - 0s 5ms/step - loss: 11.4936
Epoch 5288/10000
1/1 [==============================] - 0s 4ms/step - loss: 4.8313
Epoch 5289/10000
1/1 [==============================] - 0s 4ms/step - loss: 7.5459
Epoch 5290/10000
1/1 [==============================] - 0s 4ms/step - loss: 8.3962
Epoch 5291/10000
1/1 [==============================] - 0s 4ms/step - loss: 14.1832
Epoch 5292/10000
1/1 [==============================] - 0s 4ms/step - loss: 8.6278
Epoch 5293/100

1/1 [==============================] - 0s 4ms/step - loss: 15.0861
Epoch 5380/10000
1/1 [==============================] - 0s 4ms/step - loss: 12.0455
Epoch 5381/10000
1/1 [==============================] - 0s 4ms/step - loss: 13.7278
Epoch 5382/10000
1/1 [==============================] - 0s 4ms/step - loss: 16.3494
Epoch 5383/10000
1/1 [==============================] - 0s 4ms/step - loss: 6.4961
Epoch 5384/10000
1/1 [==============================] - 0s 4ms/step - loss: 11.9555
Epoch 5385/10000
1/1 [==============================] - 0s 4ms/step - loss: 14.6897
Epoch 5386/10000
1/1 [==============================] - 0s 4ms/step - loss: 9.8062
Epoch 5387/10000
1/1 [==============================] - 0s 4ms/step - loss: 5.9606
Epoch 5388/10000
1/1 [==============================] - 0s 5ms/step - loss: 9.5675
Epoch 5389/10000
1/1 [==============================] - 0s 4ms/step - loss: 13.7657
Epoch 5390/10000
1/1 [==============================] - 0s 4ms/step - loss: 8.8990
Epoch 5391/100

1/1 [==============================] - 0s 5ms/step - loss: 17.6922
Epoch 5478/10000
1/1 [==============================] - 0s 4ms/step - loss: 24.9337
Epoch 5479/10000
1/1 [==============================] - 0s 5ms/step - loss: 15.0423
Epoch 5480/10000
1/1 [==============================] - 0s 5ms/step - loss: 7.0053
Epoch 5481/10000
1/1 [==============================] - 0s 4ms/step - loss: 11.7461
Epoch 5482/10000
1/1 [==============================] - 0s 4ms/step - loss: 6.9982
Epoch 5483/10000
1/1 [==============================] - 0s 5ms/step - loss: 8.8618
Epoch 5484/10000
1/1 [==============================] - 0s 4ms/step - loss: 6.4144
Epoch 5485/10000
1/1 [==============================] - 0s 4ms/step - loss: 9.5654
Epoch 5486/10000
1/1 [==============================] - 0s 5ms/step - loss: 2.7476
Epoch 5487/10000
1/1 [==============================] - 0s 5ms/step - loss: 7.2877
Epoch 5488/10000
1/1 [==============================] - 0s 5ms/step - loss: 8.8858
Epoch 5489/10000


1/1 [==============================] - 0s 4ms/step - loss: 3.2928
Epoch 5576/10000
1/1 [==============================] - 0s 5ms/step - loss: 3.9908
Epoch 5577/10000
1/1 [==============================] - 0s 5ms/step - loss: 10.7005
Epoch 5578/10000
1/1 [==============================] - 0s 4ms/step - loss: 8.5002
Epoch 5579/10000
1/1 [==============================] - 0s 5ms/step - loss: 6.7592
Epoch 5580/10000
1/1 [==============================] - 0s 5ms/step - loss: 10.6424
Epoch 5581/10000
1/1 [==============================] - 0s 4ms/step - loss: 8.3905
Epoch 5582/10000
1/1 [==============================] - 0s 5ms/step - loss: 11.5544
Epoch 5583/10000
1/1 [==============================] - 0s 4ms/step - loss: 13.1446
Epoch 5584/10000
1/1 [==============================] - 0s 4ms/step - loss: 9.3713
Epoch 5585/10000
1/1 [==============================] - 0s 4ms/step - loss: 25.9407
Epoch 5586/10000
1/1 [==============================] - 0s 4ms/step - loss: 10.4597
Epoch 5587/1000

In [95]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 597ms/step
[144.989](test_target) - [[151.64508]](prediction) = 6.656080566406246


In [96]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.04590748654316014

In [97]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            train_input = train_input.fillna(0.0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            test_input = test_input.fillna(0.0)
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [98]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - IDH,Roraima - IDH Longevidade,Roraima - IDH Educacao,Roraima - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Roraima - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-0.970511,-1.141968,1.089794,-0.856005,-1.075992,-1.268873,-1.294281,1.266573,1.148205,-1.122043,...,1.308949,1.412346,-0.519434,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.405576,-0.151465,0.235656,-0.546903,1.332779,0.093630,0.153555,-0.088452,0.140893,-0.184474,...,-0.190804,-0.643250,1.398858,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.376087,1.293433,-1.325449,1.402908,-0.256787,1.175243,1.140727,-1.178121,-1.289098,1.306518,...,-1.118144,-0.769096,-0.879424,0.0,0.0,0.0,0.0,0.0,0.0,0.0


train_target:


,Roraima - Consumo de Cimento (t)
0,63.064
1,86.408
2,103.009


test_input:


...

,Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - IDH,Roraima - IDH Longevidade,Roraima - IDH Educacao,Roraima - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Roraima - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.646085,-0.836784,-1.404774,0.642293,-2.414957,0.237678,-2.61321,-1.325075,1.108405,2.635823,...,-1.482779,-2.260447,0.345514,-1.395304,-1.927921,-2.672061,2.471949,-1.925114,-2.328206,2.648601


test_target:


,Roraima - Consumo de Cimento (t)
12,144.989


1/1 [==============================] - 1s 612ms/step
Error: 6.656080566406246




[101.1805419921875,
 100.17259979248047,
 71.6328353881836,
 36.28709411621094,
 109.39332580566406,
 104.20707702636719,
 97.63934326171875,
 79.24158477783203,
 123.36321258544922,
 151.64508056640625]

In [99]:
display(targets)
display(predictions)

[100.066,
 121.914,
 120.41757142857142,
 113.28205714285713,
 105.48137142857142,
 99.723,
 83.012,
 121.12,
 151.571,
 144.989]

[101.1805419921875,
 100.17259979248047,
 71.6328353881836,
 36.28709411621094,
 109.39332580566406,
 104.20707702636719,
 97.63934326171875,
 79.24158477783203,
 123.36321258544922,
 151.64508056640625]

In [100]:
mae = mean_absolute_error(predictions, targets)
mae

24.840129913504462

In [101]:
porcentage = mae/np.mean(targets)
porcentage

0.2138485119656782